In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly_express as px
import datetime

In [2]:
df=pd.read_csv("Global_Earthquake_Data.csv")

In [3]:
df.dtypes



time                object
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

In [4]:
df

time  latitude  longitude    depth   mag magType   
0       2023-02-26T23:58:05.052Z   41.8050    79.8675   10.000  5.00      mb  \
1       2023-02-26T23:33:17.641Z   18.7420   145.4868  200.365  4.80      mb   
2       2023-02-26T21:42:14.541Z   42.0857    79.9516   10.000  4.90      mb   
3       2023-02-26T21:35:01.303Z   14.9364  -104.5563   10.000  4.60      mb   
4       2023-02-26T18:58:54.828Z   44.6730   146.5159  134.299  4.50      mb   
...                          ...       ...        ...      ...   ...     ...   
283127  1906-01-21T13:49:33.570Z   34.1750   138.0250  300.000  7.40      mw   
283128  1906-01-09T23:04:45.570Z   49.2560    18.4210   15.000  5.63      mw   
283129  1906-01-08T00:15:00.000Z   39.3000   -96.6000      NaN  4.90      fa   
283130  1906-01-06T21:31:33.220Z   53.4180   170.1000   35.000  6.58      mw   
283131  1906-01-02T04:26:27.250Z   45.3860    16.5210   15.000  5.79      mw   

          nst    gap   dmin   rms  ...                   updated   
0        46.0   91.0  1.293  0.80  ...  2023-02-27T00:11:38.040Z  \
1        67.0   85.0  5.158  0.95  ...  2023-02-26T23:58:32.040Z   
2        45.0   77.0  1.223  0.82  ...  2023-02-27T00:21:32.040Z   
3        51.0  217.0  5.661  0.57  ...  2023-02-26T22:25:29.040Z   
4       108.0   62.0  2.866  0.82  ...  2023-02-26T19:38:26.040Z   
...       ...    ...    ...   ...  ...                       ...   
283127    NaN    NaN    NaN   NaN  ...  2022-04-25T20:24:35.887Z   
283128    NaN    NaN    NaN   NaN  ...  2022-04-25T20:36:39.809Z   
283129    NaN    NaN    NaN   NaN  ...  2018-06-04T20:43:44.000Z   
283130    NaN    NaN    NaN   NaN  ...  2022-04-25T20:39:47.898Z   
283131    NaN    NaN    NaN   NaN  ...  2022-04-25T20:36:34.425Z   

                                         place        type horizontalError   
0                     77 km NNW of Aksu, China  earthquake            6.59  \
1       Pagan region, Northern Mariana Islands  earthquake           10.27   
2                                          NaN  earthquake            6.27   
3                   northern East Pacific Rise  earthquake           11.79   
4                   84 km NE of Otrada, Russia  earthquake            8.66   
...                                        ...         ...             ...   
283127               50 km SSW of ?yama, Japan  earthquake             NaN   
283128            10 km WNW of Byt?a, Slovakia  earthquake             NaN   
283129                                  Kansas  earthquake             NaN   
283130      216 km WNW of Attu Station, Alaska  earthquake             NaN   
283131              4 km WNW of Sunja, Croatia  earthquake             NaN   

       depthError  magError  magNst    status  locationSource magSource  
0           1.897     0.078    52.0  reviewed              us        us  
1           7.539     0.043   165.0  reviewed              us        us  
2           1.869     0.086    42.0  reviewed              us        us  
3           1.984     0.038   205.0  reviewed              us        us  
4           5.863     0.059    84.0  reviewed              us        us  
...           ...       ...     ...       ...             ...       ...  
283127     25.000     0.300     NaN  reviewed          iscgem    iscgem  
283128      3.500     0.650     NaN  reviewed          iscgem    iscgem  
283129        NaN       NaN     NaN  reviewed           ushis       bar  
283130      3.500     0.200     NaN  reviewed          iscgem    iscgem  
283131      4.900     0.410     NaN  reviewed          iscgem    iscgem  

[283132 rows x 22 columns]

In [5]:
df.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

In [6]:
df['time']=pd.to_datetime(df['time'])
df['time']=df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [7]:
df['updated']=pd.to_datetime(df['updated'])
df['updated']=df['updated'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [8]:
df['time']=pd.to_datetime(df['time'])


In [9]:
df['date']=df['time'].dt.strftime('%Y-%m-%d')


In [10]:
df['hour']=df['time'].dt.strftime('%H')


In [11]:
df['date']=pd.to_datetime(df['date'])


In [12]:
df['year']=df['date'].dt.strftime('%Y')
df['year']=pd.to_numeric(df['year'])


In [13]:
df=df.copy().dropna(subset=['place'])
df['country'] = df['place'].str.split(',\s*').str[-1]

In [14]:
import pycountry as pc 
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def get_country_code(country_name):
    try:
        country_code=pc.countries.search_fuzzy(country_name)[0]
        return country_code.alpha_3
    except LookupError:
        return None 

def apply_parallel(df, func, num_threads):
    results = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(func, value) for value in df]
        for future in tqdm(futures, total=len(df)):
            results.append(future.result())
    return pd.Series(results)




#df['code'] = apply_parallel(df['country'], get_country_code, num_threads=4)

In [15]:
#df.to_csv("test.csv")

In [16]:
##df=df.drop(columns='Unnamed: 0')


In [17]:
df['magType'].value_counts()

magType
mb       212615
mw        25274
mwc       20322
mww        9374
ms         4454
mwb        3136
mwr        2869
ml         2846
md          797
m           302
mh           87
uk           68
fa           56
mblg         19
mlr          14
lg           11
mwp           7
Ml            6
Md            6
mc            5
Mb            4
mb_lg         3
ms_20         3
mlg           3
ma            2
Mi            1
Name: count, dtype: int64

In [18]:
df['date']=pd.to_datetime(df['date'])
df['month']=df['date'].dt.strftime('%m')
df['month']=pd.to_numeric(df['month'])


In [19]:
def convert_mag(magnitude, magtype):
    if magtype == 'mw':
        return (magnitude-0.8 )
    elif magtype=='mww':
        return(magnitude-0.4)
    elif magtype=='mwc':
        return(magnitude-0.6)
    elif magtype=='ms':
        return(magnitude+0.7)
    elif magtype=='mwr':
        return(magnitude-0.6)
    elif magtype=='mwb':
        return(magnitude-0.7)
    elif magtype=='m':
        return(magnitude+0.7)
    elif magtype=='m':
        return(magnitude-0.3)
    elif magtype=='mwp':
        return(magnitude-0.5)
    elif magtype=='Ml':
        return(magnitude-0.5)
    elif magtype=='Md':
        return(magnitude+0.6)
    elif magtype=='lg':
        return(magnitude+1)
    else:
        return magnitude
    
    
    
    
df['magnitude'] = df.apply(lambda row: convert_mag(row['mag'], row['magType']), axis=1).round(2)

In [20]:
df['year'].value_counts()


year
2011    9571
2021    8871
2014    8306
2005    7874
2018    7550
        ... 
1910      61
1909      59
1912      49
1911      44
1908      33
Name: count, Length: 118, dtype: int64

In [21]:
df=df.sort_values('year').reset_index()

In [22]:
df=df.drop(columns='index')

In [23]:
df1=pd.read_csv("S0925231220315137.csv")

In [24]:
df = df.merge(df1[['country','alpha-3','region']], left_on='country',right_on='country')

In [25]:
df

time  latitude  longitude  depth   mag magType    nst   
0      1906-01-02 04:26:27   45.3860    16.5210   15.0  5.79      mw    NaN  \
1      1907-05-18 00:59:58   43.5890    16.7480   15.0  5.22      mw    NaN   
2      1909-10-08 09:59:11   45.4640    16.0900   15.0  5.96      mw    NaN   
3      1916-03-12 03:24:00   45.1690    14.8700   15.0  5.90      mw    NaN   
4      1916-07-14 20:27:01   44.7020    15.2020   15.0  5.77      mw    NaN   
...                    ...       ...        ...    ...   ...     ...    ...   
202237 2012-08-16 02:38:11  -17.5450  -149.1580   16.4  4.80      mb  195.0   
202238 2017-09-23 18:32:09  -12.0506  -138.4626   10.0  4.50      mb    NaN   
202239 2017-01-18 21:48:22   19.6049    10.6491   10.0  4.60      mb    NaN   
202240 2018-10-26 12:49:48   23.0794   -70.4684   10.0  4.50      mb    NaN   
202241 2019-12-19 15:25:58    1.8052     8.1840   10.0  5.50     mww    NaN   

          gap    dmin   rms  ... locationSource  magSource       date hour   
0         NaN     NaN   NaN  ...         iscgem     iscgem 1906-01-02   04  \
1         NaN     NaN   NaN  ...      iscgemsup  iscgemsup 1907-05-18   00   
2         NaN     NaN   NaN  ...         iscgem     iscgem 1909-10-08   09   
3         NaN     NaN   NaN  ...         iscgem     iscgem 1916-03-12   03   
4         NaN     NaN   NaN  ...         iscgem     iscgem 1916-07-14   20   
...       ...     ...   ...  ...            ...        ...        ...  ...   
202237   45.5     NaN  0.74  ...             us         us 2012-08-16   02   
202238  180.0   3.582  0.89  ...             us         us 2017-09-23   18   
202239   68.0  16.108  0.78  ...             us         us 2017-01-18   21   
202240   40.0   1.677  1.12  ...             us         us 2018-10-26   12   
202241   34.0   2.777  0.73  ...             us         us 2019-12-19   15   

        year                country  month  magnitude  alpha-3    region  
0       1906                Croatia      1       4.99      HRV    Europe  
1       1907                Croatia      5       4.42      HRV    Europe  
2       1909                Croatia     10       5.16      HRV    Europe  
3       1916                Croatia      3       5.10      HRV    Europe  
4       1916                Croatia      7       4.97      HRV    Europe  
...      ...                    ...    ...        ...      ...       ...  
202237  2012       French Polynesia      8       4.80      PYF   Oceania  
202238  2017       French Polynesia      9       4.50      PYF   Oceania  
202239  2017                  Niger      1       4.60      NER    Africa  
202240  2018                Bahamas     10       4.50      BHS  Americas  
202241  2019  Sao Tome and Principe     12       5.10      STP    Africa  

[202242 rows x 30 columns]

In [26]:
df=df.sort_values('year')

In [27]:
(df[['mag','country','time','magnitude','magType']].loc[df['country']=='Romania']).head(20)


mag  country                time  magnitude magType
190726  6.36  Romania 1916-01-26 07:38:03       5.56      mw
190727  5.42  Romania 1919-04-18 06:20:10       4.62      mw
190728  5.79  Romania 1929-11-01 06:57:24       4.99      mw
190729  5.45  Romania 1932-05-27 10:42:01       4.65      mw
190730  6.72  Romania 1934-03-29 20:06:50       5.92      mw
190731  5.17  Romania 1936-09-06 04:48:57       4.37      mw
190732  7.74  Romania 1940-11-10 01:39:11       6.94      mw
190733  6.63  Romania 1940-10-22 06:37:01       5.83      mw
190734  6.70  Romania 1945-09-07 15:48:26       5.90      mw
190735  4.92  Romania 1950-05-10 02:08:47       4.12      mw
190736  5.35  Romania 1959-05-31 12:15:46       4.55      mw
190737  5.28  Romania 1959-05-27 20:38:28       4.48      mw
190738  5.36  Romania 1960-01-04 12:52:00       4.56      mw
190739  5.18  Romania 1969-04-12 20:38:42       4.38      mw
190742  4.90  Romania 1973-10-23 10:50:58       4.90      mb
190741  5.60  Romania 1973-08-20 15:18:28       5.60      mb
190740  4.60  Romania 1973-09-07 19:37:51       4.60      mb
190743  5.10  Romania 1974-07-17 05:09:23       5.10      mb
190744  5.60  Romania 1974-04-17 01:31:33       5.60      mb
190747  4.60  Romania 1975-02-08 08:21:18       4.60      mb

In [28]:
import flagpy as fp
def get_flag(country):
    try:
        return fp.get_flag_img(country)
    except LookupError:
        return None
def apply_parallel(df, func, num_threads):
    results = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(func, value) for value in df]
        for future in tqdm(futures, total=len(df)):
            results.append(future.result())
    return pd.Series(results)




#df['flag'] = apply_parallel(df['country'], get_flag, num_threads=4)


In [29]:
df=df.loc[df['year']>=1960]

In [30]:
px.histogram(df['year'], nbins=25, histnorm='probability density',opacity=0.4).update_xaxes(title_text="Year").update_layout(
    title_text="Distributia totala a numarului de cutremure",title_x=0.5,font_family="Courier New",
    font_color="purple",
    title_font_family="Times New Roman",
    title_font_color="red",
    legend_title_font_color="green")


In [31]:
japan=df.loc[df['year']>=2010]
japan=df.groupby('hour').size().reset_index(name='frequency')
px.line(japan,'hour','frequency',text='frequency').update_traces(textposition='top center')

In [32]:
df['dsec']=df['dmin']*60

In [33]:
def round_to_nearest(x, base=25):
    try:
        return base * round(x/base)
    except ValueError:
        None


# apply the rounding function to the "values" column
df['rounded_depth'] = df['depth'].apply(lambda x: round_to_nearest(x, base=25))

x=df[['rounded_depth','dsec','magnitude']].groupby('rounded_depth').mean(['dsec','magnitude']).reset_index()



In [34]:
x

rounded_depth        dsec  magnitude
0               0  176.575667   4.749707
1              25  148.690257   4.824245
2              50  142.687781   4.808725
3              75  146.809490   4.770155
4             100  131.674860   4.753807
5             125  135.484302   4.756174
6             150  148.252413   4.760954
7             175  163.284740   4.749236
8             200  161.685188   4.748537
9             225  173.084444   4.776915
10            250  187.627967   4.747562
11            275  190.057009   4.785695
12            300  221.244507   4.733499
13            325  203.892000   4.742099
14            350  188.296452   4.754437
15            375  201.472286   4.795329
16            400  236.394545   4.789538
17            425  209.609333   4.792152
18            450  214.060909   4.808086
19            475  246.032400   4.784082
20            500  259.539808   4.730477
21            525  230.382762   4.786278
22            550  232.770217   4.803120
23            575  221.438276   4.858512
24            600  219.244818   4.841958
25            625  208.704516   4.881536
26            650  225.158182   4.873891
27            675  192.260000   4.846032
28            700         NaN   4.927273

In [35]:
px.line(x,x='rounded_depth',y='dsec')


In [36]:
from sklearn.preprocessing import StandardScaler

def scale_columns(df, columns):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df[columns])
    df_scaled = pd.DataFrame(df_scaled, columns=columns)
    df[columns] = df_scaled
    return df


In [37]:
x = scale_columns(x, ['dsec', 'magnitude'])


In [38]:
x

rounded_depth      dsec  magnitude
0               0 -0.477965  -0.883835
1              25 -1.268127   0.658750
2              50 -1.438214   0.337561
3              75 -1.321421  -0.460666
4             100 -1.750276  -0.798988
5             125 -1.642332  -0.749992
6             150 -1.280534  -0.651080
7             175 -0.854577  -0.893585
8             200 -0.899902  -0.908052
9             225 -0.576892  -0.320757
10            250 -0.164786  -0.928233
11            275 -0.095957  -0.139051
12            300  0.787774  -1.219268
13            325  0.296072  -1.041279
14            350 -0.145844  -0.785944
15            375  0.227507   0.060323
16            400  1.217066  -0.059531
17            425  0.458079  -0.005427
18            450  0.584219   0.324342
19            475  1.490164  -0.172443
20            500  1.872911  -1.281796
21            525  1.046716  -0.126993
22            550  1.114367   0.221562
23            575  0.793264   1.367905
24            600  0.731110   1.025317
25            625  0.432440   1.844409
26            650  0.898672   1.686192
27            675 -0.033533   1.109630
28            700       NaN   2.790932

In [39]:
import plotly.graph_objects as go
trace1 = go.Scatter(x=x['rounded_depth'][:2],
                    y=x['dsec'][:2],
                    mode='lines',
                    line=dict(width=1.5,color='red'),name='sec',legendgroup='group1')
trace2 = go.Scatter(x=x['rounded_depth'][:2],
                    y=x['magnitude'][:2],

                    mode='lines',
                    line=dict(width=1.5,color='blue'),name='magnitude',legendgroup='group2',)
frames = [dict(data= [dict(type='scatter',
                           x=x['rounded_depth'][:k+1],
                           y=x['dsec'][:k+1]),
                      dict(type='scatter',
                           x=x['rounded_depth'][:k+1],
                           y=x['magnitude'][:k+1]),
                     ],
               traces= [0, 1],  
              )for k  in  range(1, len(x['rounded_depth'])-1)]
layout = go.Layout(width=1400,
                   height=600,
                   showlegend=False,
                   hovermode='x unified',
                   updatemenus=[
                        dict(
                            type='buttons', showactive=False,
                            y=1.05,
                            x=1.15,
                            xanchor='right',
                            yanchor='top',
                            pad=dict(t=0, r=10),
                            buttons=[dict(label='Play',
                            method='animate',
                            args=[None, 
                                  dict(frame=dict(duration=3, 
                                                  redraw=False),
                                                  transition=dict(duration=0),
                                                  fromcurrent=True,
                                                  mode='immediate')]
                            )]
                        ),
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=[{"yaxis.type": "linear"}],
                                    label="LINEAR",
                                    method="relayout"
                                ),
                                
                            ]),
                        ),
                    ]              
                  )
layout.update(xaxis =dict(range=['0', '700'], autorange=False),
              yaxis =dict(range=[-10, 10], autorange=False));
fig = go.Figure(data=[trace1, trace2], frames=frames, layout=layout)
fig.update_traces(showlegend=True,)

fig.show()

Mape ce evidentiaza zonele afectate 

In [40]:
"""
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='magnitude', animation_frame='year',
                        zoom=1, hover_name='country',
                        color_continuous_scale=
                     [
                            [0, "rgb(255,255,255)"],        # white for magnitude 0
                            [0.1, "rgb(255,255,0)"],        # yellow for low magnitude
                            [0.3, "rgb(255,165,0)"],        # orange
                            [0.5, "rgb(255,69,0)"],         # orange-red
                            [0.7, "rgb(255,0,0)"],          # red for high magnitude
                            [1.0, "rgb(128,0,0)"]           # dark red for very high magnitude
                        ],
                        mapbox_style='open-street-map', height=1000,range_color=(0, 10),
                        #size='magnitude',size_max=10
)
                        

# Show plot
fig.show()
"""

'\nfig = px.scatter_mapbox(df, lat=\'latitude\', lon=\'longitude\', color=\'magnitude\', animation_frame=\'year\',\n                        zoom=1, hover_name=\'country\',\n                        color_continuous_scale=\n                     [\n                            [0, "rgb(255,255,255)"],        # white for magnitude 0\n                            [0.1, "rgb(255,255,0)"],        # yellow for low magnitude\n                            [0.3, "rgb(255,165,0)"],        # orange\n                            [0.5, "rgb(255,69,0)"],         # orange-red\n                            [0.7, "rgb(255,0,0)"],          # red for high magnitude\n                            [1.0, "rgb(128,0,0)"]           # dark red for very high magnitude\n                        ],\n                        mapbox_style=\'open-street-map\', height=1000,range_color=(0, 10),\n                        #size=\'magnitude\',size_max=10\n)\n                        \n\n# Show plot\nfig.show()\n'

In [41]:
df1=df.loc[df['magnitude']>=6]

In [42]:
custom_colors = [
    'rgb(255, 255, 255)',  # Level 1: White
    'rgb(173, 216, 230)',  # Level 2/3: Light Blue
    'rgb(144, 238, 144)',  # Level 5: Light Green
    'rgb(255, 165, 0)',    # Level 7: Orange
    'rgb(255, 0, 0)'       # Level 9: Red
]
px.choropleth(df1, 
              locations ='country',
              locationmode='country names',
              color="magnitude", 
              animation_frame="year",
              color_continuous_scale=custom_colors,
              featureidkey='properties.name',
              range_color=(0, 10),
              title='',
              height=800,
              width=1400
             )

In [43]:
x=df[['year','country','magnitude']].groupby(['year','country']).max('magnitude').reset_index()

In [44]:
custom_colors = [
    'rgb(255, 255, 255)',  # Level 1: White
    'rgb(173, 216, 230)',  # Level 2/3: Light Blue
    'rgb(144, 238, 144)',  # Level 5: Light Green
    'rgb(255, 165, 0)',    # Level 7: Orange
    'rgb(255, 0, 0)'       # Level 9: Red
]

px.choropleth(x, 
              locations ='country',
              locationmode='country names',
              color="magnitude", 
              animation_frame="year",
              color_continuous_scale=custom_colors,
              featureidkey='properties.name',
              range_color=(0, 10),
              title='',
              height=800
             )

In [45]:
df1=df.loc[df['type']=='nuclear explosion']

fig = px.scatter_mapbox(df1, lat='latitude', lon='longitude', color='magnitude', animation_frame='year',
                        zoom=1, hover_name='country',
                        color_continuous_scale=
                     [
                            [0, "rgb(255,255,255)"],        # white for magnitude 0
                            [0.1, "rgb(255,255,0)"],        # yellow for low magnitude
                            [0.3, "rgb(255,165,0)"],        # orange
                            [0.5, "rgb(255,69,0)"],         # orange-red
                            [0.7, "rgb(255,0,0)"],          # red for high magnitude
                            [1.0, "rgb(128,0,0)"]           # dark red for very high magnitude
                        ],
                        mapbox_style='open-street-map', height=1000,range_color=(0, 10),
                        size='magnitude',size_max=10
)
                        

# Show plot
fig.show()

In [46]:
#df1=df.copy()
#df1=df1.loc[df1['code']!=None]
country_year_freq = df.groupby(['country', 'year','alpha-3']).size().reset_index(name='frequency')



In [47]:
country_year_freq=country_year_freq.loc[country_year_freq['frequency']>=25].sort_values('year')


In [48]:

country_year_freq = country_year_freq.sort_values(['year','frequency'], ascending=[True,False])



In [49]:
from IPython.display import display

#for i in range(len(country_year_freq)):
    #df['new']=display(df['flag'][i])


In [50]:
country_year_freq = df.groupby(['country', 'year','alpha-3']).size().reset_index(name='frequency')
country_year_freq = country_year_freq.sort_values(['year','frequency'], ascending=[True,False])


In [51]:
country_year_freq['suma']=country_year_freq.groupby('country')['frequency'].cumsum()

In [52]:
country_year_freq

country  year alpha-3  frequency  suma
805               Chile  1960     CHL         41    41
2353              Japan  1960     JPN         20    20
2098          Indonesia  1960     IDN         16    16
3555   Papua New Guinea  1960     PNG         14    14
3931             Russia  1960     RUS         14    14
...                 ...   ...     ...        ...   ...
2883           Mongolia  2023     MNG          1   194
2945            Morocco  2023     MAR          1    98
4479              Syria  2023     SYR          1    17
5153          Venezuela  2023     VEN          1   435
5230  Wallis and Futuna  2023     WLF          1   698

[5340 rows x 5 columns]

In [53]:
df_top10 = country_year_freq.groupby('year').apply(lambda x: x.nlargest(20, 'suma')).reset_index(drop=True).sort_values(['year','suma'],ascending=[True,True])

fig = px.bar(df_top10, x='suma', y='alpha-3', animation_frame='year',
             range_x=[0, max(country_year_freq['suma'])+10], color='suma', range_color=(0, 35000), height=800, text='suma',hover_name='country')
fig.update_layout(xaxis=dict(range=[0, 33500]),)
fig.update_traces(textposition='inside')


fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000

fig.show()


country_year_freq

In [54]:

df_top10 = country_year_freq.groupby('year').apply(lambda x: x.nlargest(20, 'suma')).reset_index(drop=True).sort_values(['year','frequency'],ascending=[True,True])
fig = px.bar(df_top10, x='frequency', y='alpha-3', animation_frame='year',
             range_x=[0, max(country_year_freq['frequency'])+10],color='frequency',height=900,text='frequency',).update_layout( xaxis=dict(range=[0, 3500]
    )).update_xaxes(title="")
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000


fig.show()

In [55]:
x=country_year_freq.drop(columns='year')
x=x.groupby('country').sum('frequency').sort_values('frequency',ascending=False)

In [56]:
x

frequency    suma
country                                 
Indonesia                  32438  733114
Japan                      18155  402387
Papua New Guinea           15773  365055
Philippines                13580  320375
Tonga                      10789  230425
...                          ...     ...
Sao Tome and Principe          1       1
Tokelau                        1       1
Netherlands                    1       1
Ghana                          1       1
Eswatini                       1       1

[175 rows x 2 columns]

In [57]:

country_year_freq = country_year_freq.sort_values(['year','frequency'], ascending=[True,False])


In [58]:
country_year_freq

country  year alpha-3  frequency  suma
805               Chile  1960     CHL         41    41
2353              Japan  1960     JPN         20    20
2098          Indonesia  1960     IDN         16    16
3555   Papua New Guinea  1960     PNG         14    14
3931             Russia  1960     RUS         14    14
...                 ...   ...     ...        ...   ...
2883           Mongolia  2023     MNG          1   194
2945            Morocco  2023     MAR          1    98
4479              Syria  2023     SYR          1    17
5153          Venezuela  2023     VEN          1   435
5230  Wallis and Futuna  2023     WLF          1   698

[5340 rows x 5 columns]

In [59]:
indonesia=country_year_freq[country_year_freq['country'].isin(['Indonesia'])]
japan=country_year_freq[country_year_freq['country'].isin(['Japan'])]
papua=country_year_freq[country_year_freq['country'].isin(['Papua New Guinea'])]
russia=country_year_freq[country_year_freq['country'].isin(['Russia'])]
phil=country_year_freq[country_year_freq['country'].isin(['Philippines'])]

In [60]:
phil

country  year alpha-3  frequency   suma
3683  Philippines  1960     PHL          4      4
3684  Philippines  1961     PHL          3      7
3685  Philippines  1962     PHL          7     14
3686  Philippines  1963     PHL          8     22
3687  Philippines  1964     PHL          8     30
...           ...   ...     ...        ...    ...
3742  Philippines  2019     PHL        548  12411
3743  Philippines  2020     PHL        357  12768
3744  Philippines  2021     PHL        348  13116
3745  Philippines  2022     PHL        415  13531
3746  Philippines  2023     PHL         49  13580

[64 rows x 5 columns]

In [61]:
import plotly.graph_objects as go
trace1 = go.Scatter(x=indonesia['year'][:2],
                    y=indonesia['frequency'][:2],
                    mode='lines',
                    line=dict(width=1.5,color='red'),name='Indonesia',legendgroup='group1')
trace2 = go.Scatter(x=japan['year'][:2],
                    y=japan['frequency'][:2],

                    mode='lines',
                    line=dict(width=1.5,color='blue'),name='Japan',legendgroup='group2',)
trace3 = go.Scatter(x=papua['year'][:2],
                    y=papua['frequency'][:2],
                    mode='lines',
                    line=dict(width=1.5,color='purple'),name='Papua New Guinea',legendgroup='group3',)
trace4 = go.Scatter(x=phil['year'][:2],
                    y=phil['frequency'][:2],
                    mode='lines',
                    line=dict(width=1.5,color='orange'),name='Philippines',legendgroup='group4', )
trace5 = go.Scatter(x=russia['year'][:2],
                    y=russia['frequency'][:2],
                    mode='lines',
                    line=dict(width=1.5,color='green'),name='Russia',legendgroup='group5', )
frames = [dict(data= [dict(type='scatter',
                           x=indonesia['year'][:k+1],
                           y=indonesia['frequency'][:k+1]),
                      dict(type='scatter',
                           x=japan['year'][:k+1],
                           y=japan['frequency'][:k+1]),
                      dict(type='scatter',
                           x=papua['year'][:k+1],
                           y=papua['frequency'][:k+1]),
                        dict(type='scatter',
                           x=phil['year'][:k+1],
                           y=phil['frequency'][:k+1]),
                      dict(type='scatter',
                           x=russia['year'][:k+1],
                           y=russia['frequency'][:k+1]),
                     ],
               traces= [0, 1, 2, 3, 4],  
              )for k  in  range(1, len(indonesia)+1)]
layout = go.Layout(width=1400,
                   height=600,
                   showlegend=True,
                   
                   hovermode='x unified',
                   updatemenus=[
                        dict(
                            type='buttons', showactive=False,
                            y=1.05,
                            x=1.15,
                            xanchor='right',
                            yanchor='top',
                            pad=dict(t=0, r=10),
                            buttons=[dict(label='Play',
                            method='animate',
                            args=[None, 
                                  dict(frame=dict(duration=3, 
                                                  redraw=False),
                                                  transition=dict(duration=0),
                                                  fromcurrent=True,
                                                  mode='immediate')]
                            )]
                        ),
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=[{"yaxis.type": "linear"}],
                                    label="LINEAR",
                                    method="relayout"
                                ),
                                dict(
                                    args=[{"yaxis.type": "log"}],
                                    label="LOG",
                                    method="relayout"
                                )
                            ]),
                        ),
                    ] 
                               
                  )
layout.update(xaxis =dict(range=['1955', '2025'], autorange=False),
              yaxis =dict(range=[0, 3800], autorange=False));
fig = go.Figure(data=[trace1, trace2, trace3, trace4,trace5], frames=frames, layout=layout)
fig.update_traces(showlegend=True,)

fig.show()

In [62]:
df[['year','country','magnitude']].groupby(['year','country']).max('magnitude').reset_index()

year              country  magnitude
0     1960          Afghanistan       4.88
1     1960              Albania       5.42
2     1960              Algeria       4.52
3     1960       American Samoa       5.03
4     1960  Antigua and Barbuda       4.97
...    ...                  ...        ...
5435  2023                Tonga       5.30
5436  2023               Turkey       7.40
5437  2023              Vanuatu       6.60
5438  2023            Venezuela       4.50
5439  2023    Wallis and Futuna       4.60

[5440 rows x 3 columns]

In [63]:
country_year_freq = df.groupby(['country', 'year','alpha-3','region']).size().reset_index(name='frequency')
country_year_freq=country_year_freq.loc[country_year_freq['year']>=1980]


In [64]:
country_year_freq

country  year alpha-3  region  frequency
19    Afghanistan  1980     AFG    Asia         32
20    Afghanistan  1981     AFG    Asia         36
21    Afghanistan  1982     AFG    Asia         41
22    Afghanistan  1983     AFG    Asia         55
23    Afghanistan  1984     AFG    Asia         79
...           ...   ...     ...     ...        ...
5326     Zimbabwe  2016     ZWE  Africa          2
5327     Zimbabwe  2017     ZWE  Africa          1
5328     Zimbabwe  2018     ZWE  Africa          3
5329     Zimbabwe  2020     ZWE  Africa          2
5330     Zimbabwe  2022     ZWE  Africa          1

[4112 rows x 5 columns]

In [65]:
country_year_freq=country_year_freq.groupby(['year','region']).sum('frequency').reset_index()

In [66]:
country_year_freq

year    region  frequency
0    1980    Africa         61
1    1980  Americas        435
2    1980      Asia       1108
3    1980    Europe        271
4    1980   Oceania        675
..    ...       ...        ...
215  2023    Africa          9
216  2023  Americas        103
217  2023      Asia        443
218  2023    Europe         40
219  2023   Oceania        181

[220 rows x 3 columns]

In [67]:
import plotly.graph_objects as go
fig = go.Figure()
for region in country_year_freq['region'].unique():
    fig.add_traces(go.Scatter(
        x = country_year_freq.query(f'region == "{region}"')['year'],
        y = country_year_freq.query(f'region == "{region}"')['frequency'],
        fill = 'tozeroy',
        mode = 'lines',
        name = region,
        
    ))
    
fig.update_layout(
    # Set the name of the map
   
    font=dict(
       family='Serif',
       size=14, 
       color='black'
       
    )
    
)
fig.update_xaxes(title_text="Year").update_layout(
    title_text="Evolutia anuala a numarului de cutremure pentru fiecare continent",title_x=0.5,font_family="Courier New",
    font_color="purple",
    title_font_family="Times New Roman",
    title_font_color="red",
    legend_title_font_color="green",
    yaxis_title="frequency"
   
    )

fig.show()

In [68]:
df['dsec']=df['dmin']*60

In [69]:
country_year_freq = df.groupby(['country','month','date','year','alpha-3','magnitude']).size().reset_index(name='frequency')

japan=country_year_freq.loc[country_year_freq['country']=='Japan']

In [70]:
japan

country  month       date  year alpha-3  magnitude  frequency
67256   Japan      1 1960-01-31  1960     JPN       5.53          1
67257   Japan      1 1961-01-16  1961     JPN       5.45          1
67258   Japan      1 1961-01-16  1961     JPN       5.53          1
67259   Japan      1 1961-01-16  1961     JPN       5.89          1
67260   Japan      1 1961-01-16  1961     JPN       6.23          1
...       ...    ...        ...   ...     ...        ...        ...
81321   Japan     12 2022-12-26  2022     JPN       4.50          1
81322   Japan     12 2022-12-27  2022     JPN       4.60          1
81323   Japan     12 2022-12-29  2022     JPN       4.50          1
81324   Japan     12 2022-12-29  2022     JPN       5.10          1
81325   Japan     12 2022-12-31  2022     JPN       4.50          1

[14070 rows x 7 columns]

In [71]:
japan['magnitude'].unique()

array([5.53, 5.45, 5.89, 6.23, 6.4 , 5.38, 5.24, 5.16, 5.85, 4.8 , 5.31,
       4.76, 4.5 , 5.23, 4.97, 5.14, 5.79, 4.9 , 4.7 , 5.1 , 6.19, 5.6 ,
       4.41, 5.03, 4.79, 4.75, 4.6 , 5.  , 5.7 , 5.2 , 6.8 , 6.5 , 5.9 ,
       5.3 , 5.4 , 7.3 , 5.5 , 6.7 , 6.1 , 6.6 , 6.9 , 7.6 , 7.1 , 6.3 ,
       6.  , 4.4 , 4.2 , 5.8 , 4.3 , 6.2 , 3.9 , 4.  , 4.1 , 5.27, 5.39,
       6.74, 5.28, 4.89, 5.25, 5.04, 4.91, 4.82, 5.15, 5.21, 5.36, 4.67,
       5.46, 4.69, 6.36, 7.2 , 5.05, 7.15, 5.01, 5.09, 5.61, 6.21, 4.68,
       4.73, 4.87, 5.13, 4.74, 4.71, 4.93, 5.07, 5.44, 7.  , 7.4 , 8.7 ,
       4.84, 4.96, 5.11, 5.17, 5.19, 5.95, 6.22, 5.18, 4.77, 4.81, 5.75,
       4.99, 5.42, 4.98, 4.86, 5.86, 5.29, 5.66, 5.26, 5.33, 5.55, 5.34,
       4.85, 5.22, 5.72, 5.63, 5.65, 6.32, 4.78, 4.72, 5.35, 5.56, 4.83,
       8.4 , 4.95, 5.08, 4.47, 4.92, 4.94, 5.69, 5.74, 5.12, 4.36, 4.64,
       5.02, 5.06, 6.03, 5.48, 5.92, 4.66, 5.59, 7.76, 5.71, 5.93, 5.37,
       5.68])

In [72]:
japan=japan.loc[(japan['year'] >= 2010)] 
japan=japan.loc[(japan['year'] <=2011)] 


In [73]:
japan

country  month       date  year alpha-3  magnitude  frequency
67965   Japan      1 2010-01-07  2010     JPN        4.2          1
67966   Japan      1 2010-01-12  2010     JPN        4.3          1
67967   Japan      1 2010-01-14  2010     JPN        4.4          1
67968   Japan      1 2010-01-15  2010     JPN        5.1          1
67969   Japan      1 2010-01-16  2010     JPN        3.9          1
...       ...    ...        ...   ...     ...        ...        ...
80928   Japan     12 2011-12-24  2011     JPN        4.8          1
80929   Japan     12 2011-12-25  2011     JPN        4.5          1
80930   Japan     12 2011-12-25  2011     JPN        4.9          1
80931   Japan     12 2011-12-27  2011     JPN        4.6          1
80932   Japan     12 2011-12-30  2011     JPN        4.7          3

[1295 rows x 7 columns]

In [74]:
japan['magnitude'].max()

8.7

In [75]:
japan.sort_values('date',ascending=True)

country  month       date  year alpha-3  magnitude  frequency
67965   Japan      1 2010-01-07  2010     JPN        4.2          1
67966   Japan      1 2010-01-12  2010     JPN        4.3          1
67967   Japan      1 2010-01-14  2010     JPN        4.4          1
67968   Japan      1 2010-01-15  2010     JPN        5.1          1
67969   Japan      1 2010-01-16  2010     JPN        3.9          1
...       ...    ...        ...   ...     ...        ...        ...
80928   Japan     12 2011-12-24  2011     JPN        4.8          1
80929   Japan     12 2011-12-25  2011     JPN        4.5          1
80930   Japan     12 2011-12-25  2011     JPN        4.9          1
80931   Japan     12 2011-12-27  2011     JPN        4.6          1
80932   Japan     12 2011-12-30  2011     JPN        4.7          3

[1295 rows x 7 columns]

In [76]:
japan=japan[['frequency', 'magnitude', 'month']].groupby('month').agg({'frequency': 'sum', 'magnitude': 'mean'}).reset_index()
japan

month  frequency  magnitude
0       1         30   4.590000
1       2         57   4.589130
2       3       1978   4.930435
3       4        427   4.718785
4       5        250   4.673723
5       6        164   4.660345
6       7        169   4.669912
7       8        140   4.667568
8       9        124   4.682292
9      10         95   4.680769
10     11         93   4.630556
11     12         79   4.620968

In [77]:

indo=country_year_freq.loc[country_year_freq['country']=='Indonesia']
x=indo.loc[(country_year_freq['year'] >= 2004)] 
indo=x.loc[(country_year_freq['year'] <2006)] 
indonesia=indo[['frequency', 'magnitude', 'month']].groupby('month').agg({'frequency': 'sum', 'magnitude': 'mean'}).reset_index()


In [78]:
indonesia

month  frequency  magnitude
0       1        730   4.675714
1       2        235   4.732278
2       3        392   4.740523
3       4        576   4.674477
4       5        260   4.622807
5       6        190   4.645865
6       7        196   4.626174
7       8        152   4.612000
8       9        147   4.606667
9      10        156   4.620472
10     11        152   4.694444
11     12        389   4.730313

In [79]:

chi=country_year_freq.loc[country_year_freq['country']=='Chile']
y=chi.loc[(chi['year'] >= 2009)] 
chi=y.loc[(chi['year'] <2011)] 
chi['magnitude'].max()


8.4

In [80]:
chile=chi[['frequency', 'magnitude', 'month']].groupby('month').agg({'frequency': 'sum', 'magnitude': 'mean'}).reset_index()


In [81]:
chile

month  frequency  magnitude
0       1         21   4.819048
1       2        393   5.118679
2       3        517   4.839487
3       4         97   4.768750
4       5         79   4.685507
5       6         54   4.669231
6       7         45   4.831707
7       8         33   4.738710
8       9         38   4.641176
9      10         40   4.771795
10     11         29   4.722222
11     12         33   4.725806

In [82]:
y.loc[y['magnitude']==y['magnitude'].max()]

country  month       date  year alpha-3  magnitude  frequency
8756   Chile      2 2010-02-27  2010     CHL        8.4          1

In [83]:
bins=np.linspace(min(df['depth']),max(df['depth']),10)
bins

array([ -4.        ,  74.32222222, 152.64444444, 230.96666667,
       309.28888889, 387.61111111, 465.93333333, 544.25555556,
       622.57777778, 700.9       ])

In [84]:
corr=df.drop(columns=['time','magType','id','net','place','type','status','locationSource','magSource','date','country','alpha-3','updated','region'])

In [85]:
from scipy import stats
corr.corr()

latitude  longitude     depth       mag       nst       gap   
latitude         1.000000   0.299966 -0.170119 -0.036080  0.207063 -0.055823  \
longitude        0.299966   1.000000 -0.172358  0.002577 -0.070079 -0.049885   
depth           -0.170119  -0.172358  1.000000 -0.014660  0.074088 -0.093034   
mag             -0.036080   0.002577 -0.014660  1.000000  0.720244 -0.468180   
nst              0.207063  -0.070079  0.074088  0.720244  1.000000 -0.527614   
gap             -0.055823  -0.049885 -0.093034 -0.468180 -0.527614  1.000000   
dmin            -0.041499  -0.009987  0.044627 -0.028710 -0.047544  0.059729   
rms             -0.064787  -0.061125 -0.059342  0.084777 -0.082984 -0.034692   
horizontalError -0.175798  -0.045531  0.269961 -0.168277 -0.176561  0.293160   
depthError      -0.072119   0.040789  0.144358 -0.086522 -0.301862  0.219055   
magError        -0.059311   0.005335 -0.062110  0.274839 -0.557766  0.430388   
magNst           0.147600  -0.082229  0.063874  0.207159  0.783059 -0.275712   
hour            -0.004832   0.004225  0.004814 -0.004081 -0.017847  0.009252   
year            -0.087906  -0.006266 -0.062598 -0.248607  0.068810 -0.109549   
month            0.007611  -0.001275  0.011803  0.003534 -0.007759 -0.012375   
magnitude       -0.044736   0.021883 -0.006118  0.782293  0.480681 -0.325062   
dsec            -0.041499  -0.009987  0.044627 -0.028710 -0.047544  0.059729   
rounded_depth   -0.168676  -0.169169  0.997449 -0.008794  0.081319 -0.096964   

                     dmin       rms  horizontalError  depthError  magError   
latitude        -0.041499 -0.064787        -0.175798   -0.072119 -0.059311  \
longitude       -0.009987 -0.061125        -0.045531    0.040789  0.005335   
depth            0.044627 -0.059342         0.269961    0.144358 -0.062110   
mag             -0.028710  0.084777        -0.168277   -0.086522  0.274839   
nst             -0.047544 -0.082984        -0.176561   -0.301862 -0.557766   
gap              0.059729 -0.034692         0.293160    0.219055  0.430388   
dmin             1.000000 -0.075306         0.412829   -0.008605  0.055646   
rms             -0.075306  1.000000        -0.108609    0.147352 -0.041223   
horizontalError  0.412829 -0.108609         1.000000    0.246618  0.182330   
depthError      -0.008605  0.147352         0.246618    1.000000  0.333229   
magError         0.055646 -0.041223         0.182330    0.333229  1.000000   
magNst          -0.021242 -0.157772        -0.081641   -0.226858 -0.540973   
hour             0.021380  0.022271         0.003723    0.011889 -0.008044   
year            -0.024147 -0.355738        -0.008598   -0.295599 -0.643020   
month           -0.028780  0.008272        -0.010620   -0.016977  0.003966   
magnitude        0.036335  0.083047        -0.058514   -0.090733  0.021679   
dsec             1.000000 -0.075306         0.412829   -0.008605  0.055646   
rounded_depth    0.037542 -0.060982         0.265670    0.156935 -0.054280   

                   magNst      hour      year     month  magnitude      dsec   
latitude         0.147600 -0.004832 -0.087906  0.007611  -0.044736 -0.041499  \
longitude       -0.082229  0.004225 -0.006266 -0.001275   0.021883 -0.009987   
depth            0.063874  0.004814 -0.062598  0.011803  -0.006118  0.044627   
mag              0.207159 -0.004081 -0.248607  0.003534   0.782293 -0.028710   
nst              0.783059 -0.017847  0.068810 -0.007759   0.480681 -0.047544   
gap             -0.275712  0.009252 -0.109549 -0.012375  -0.325062  0.059729   
dmin            -0.021242  0.021380 -0.024147 -0.028780   0.036335  1.000000   
rms             -0.157772  0.022271 -0.355738  0.008272   0.083047 -0.075306   
horizontalError -0.081641  0.003723 -0.008598 -0.010620  -0.058514  0.412829   
depthError      -0.226858  0.011889 -0.295599 -0.016977  -0.090733 -0.008605   
magError        -0.540973 -0.008044 -0.643020  0.003966   0.021679  0.055646   
magNst           1.000000 -0.005532  0.344244 -0.0139

In [86]:
px.imshow(corr.corr(),text_auto=True,height=800,width=900)

In [87]:
indo

country  month       date  year alpha-3  magnitude  frequency
38734  Indonesia      1 2004-01-01  2004     IDN        4.9          1
38735  Indonesia      1 2004-01-01  2004     IDN        5.1          1
38736  Indonesia      1 2004-01-02  2004     IDN        4.5          1
38737  Indonesia      1 2004-01-03  2004     IDN        4.5          1
38738  Indonesia      1 2004-01-03  2004     IDN        4.7          1
...          ...    ...        ...   ...     ...        ...        ...
62800  Indonesia     12 2005-12-29  2005     IDN        4.5          2
62801  Indonesia     12 2005-12-29  2005     IDN        4.6          2
62802  Indonesia     12 2005-12-29  2005     IDN        4.7          1
62803  Indonesia     12 2005-12-30  2005     IDN        4.6          2
62804  Indonesia     12 2005-12-31  2005     IDN        4.8          1

[1853 rows x 7 columns]

In [88]:
import plotly.graph_objects as go

trace1 = go.Scatter(x=indonesia['month'][:2],
                    y=indonesia['frequency'][:2],
                    mode='lines',
                    line=dict(width=1.5, color='red'),
                    name='Indonesia',
                    legendgroup='group1')

trace2 = go.Scatter(x=japan['month'][:2],
                    y=japan['frequency'][:2],
                    mode='lines',
                    line=dict(width=1.5, color='blue'),
                    name='Japan',
                    legendgroup='group2')

trace3 = go.Scatter(x=chile['month'][:2],
                    y=chile['frequency'][:2],
                    mode='lines',
                    line=dict(width=1.5, color='purple'),
                    name='Chile',
                    legendgroup='group3')

frames = [dict(data=[
                dict(type='scatter', x=indonesia['month'][:k+1], y=indonesia['frequency'][:k+1]),
                dict(type='scatter', x=japan['month'][:k+1], y=japan['frequency'][:k+1]),
                dict(type='scatter', x=chile['month'][:k+1], y=chile['frequency'][:k+1])
            ],
            traces=[0, 1, 2]) for k in range(1, len(indonesia)+1)]

layout = go.Layout(
    width=1400,
    height=600,
    showlegend=True,
    hovermode='x unified',
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            y=1.05,
            x=1.15,
            xanchor='right',
            yanchor='top',
            pad=dict(t=0, r=10),
            buttons=[dict(label='Play',
                          method='animate',
                          args=[None,
                                dict(frame=dict(duration=3,
                                                redraw=False),
                                                transition=dict(duration=0),
                                                fromcurrent=True,
                                                mode='immediate')])
                     ]
        ),
        
    ]
)

layout.update(
    xaxis=dict(range=['1', '13'], autorange=False),
    yaxis=dict(range=[0, 4], type='log')
)

fig = go.Figure(data=[trace1, trace2, trace3], frames=frames, layout=layout)
fig.update_traces(showlegend=True)

fig.show()


In [89]:
import plotly.graph_objects as go

trace1 = go.Scatter(x=indonesia['month'][:2],
                    y=indonesia['magnitude'][:2],
                    mode='lines',
                    line=dict(width=1.5, color='red'),
                    name='Indonesia',
                    legendgroup='group1')

trace2 = go.Scatter(x=japan['month'][:2],
                    y=japan['magnitude'][:2],
                    mode='lines',
                    line=dict(width=1.5, color='blue'),
                    name='Japan',
                    legendgroup='group2')

trace3 = go.Scatter(x=chile['month'][:2],
                    y=chile['magnitude'][:2],
                    mode='lines',
                    line=dict(width=1.5, color='purple'),
                    name='Chile',
                    legendgroup='group3')

frames = [dict(data=[
                dict(type='scatter', x=indonesia['month'][:k+1], y=indonesia['magnitude'][:k+1]),
                dict(type='scatter', x=japan['month'][:k+1], y=japan['magnitude'][:k+1]),
                dict(type='scatter', x=chile['month'][:k+1], y=chile['magnitude'][:k+1])
            ],
            traces=[0, 1, 2]) for k in range(1, len(indonesia)+1)]

layout = go.Layout(
    width=1400,
    height=600,
    showlegend=True,
    hovermode='x unified',
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            y=1.05,
            x=1.15,
            xanchor='right',
            yanchor='top',
            pad=dict(t=0, r=10),
            buttons=[dict(label='Play',
                          method='animate',
                          args=[None,
                                dict(frame=dict(duration=3,
                                                redraw=False),
                                                transition=dict(duration=0),
                                                fromcurrent=True,
                                                mode='immediate')])
                     ]
        ),
        
    ]
)

layout.update(
    xaxis=dict(range=['1', '13'], autorange=False),
    yaxis=dict(range=[4, 6])
)

fig = go.Figure(data=[trace1, trace2, trace3], frames=frames, layout=layout)
fig.update_traces(showlegend=True)

fig.show()
